Try nmf with all non negative features with grid search of c 


In [1]:
import pandas as pd


import matplotlib.pyplot as plt
from sklearn.decomposition import NMF 
from sklearn.preprocessing import Normalizer ,MaxAbsScaler
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_csv('../data/final_per_90_and_pAdj.csv')

In [ ]:
example 

a should be x really 

A = tfidf_vectorizer.transform(texts)
W = nmf.components_
H = nmf.transform(A)
A = 301 x 5000
W = 30 x 5000
H = 301 x 30

# Get the residuals for each document
r = np.zeros(A.shape[0])
for row in range(A.shape[0]):
    r[row] = np.linalg.norm(A[row, :] - H[row, :].dot(W), 'fro')
# Add the residuals to the df
df_topics['resid'] = r

In [ ]:
## other example to show 

#@title Run NMF
documents would be rows in df , no_top_words is int like 3 , no top doc is int like 2 

def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([ (feature_names[i] + " (" + str(topic[i].round(2)) + ")")
          for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(str(doc_index) + ". " + documents[doc_index])

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# Run NMF
nmf_model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
nmf_W = nmf_model.transform(tfidf)
nmf_H = nmf_model.components_

print("NMF Topics")
display_topics(nmf_H, nmf_W, tfidf_feature_names, documents, no_top_words, no_top_documents)
print("--------------")
NMF Topics
Topic 0:
trees (0.93) graph (0.7) minors (0.43) survey (0.13)
7. Graph minors IV: Widths of trees and quasi-ordering
6. The intersection graph of paths in trees
5. The generation of random, binary, unordered trees
Topic 1:
time (0.58) response (0.58) user (0.54) survey (0.26)
4. Relation of user-perceived response time to error measurement
1. A survey of user opinion of computer system response time
2. The EPS user interface management system


In [6]:
# ## dont need club for NMF 

# clubs = df['Club']
# df.drop(columns=['Club'],inplace=True)
# missing = [df.isnull().sum(),100*df.isnull().mean()]
# m = pd.DataFrame(missing)
# m = m.T
# m.columns=['count', '%']
# m = m.sort_values(by='count',ascending=False)
# m.head()
# df['Base Salary'] = df['Base Salary'].str.replace('$','')
# df['Base Salary'] = df['Base Salary'].str.replace(',','')
# df['Base Salary'] = pd.to_numeric(df['Base Salary'])
# df[['Base Salary']].info()
# cols = ['Base Salary']
# for col in cols:
#     print(df[col].mean())
#     df[col] = df[col].fillna(df[col].mean())

missing = [df.isnull().sum(),100*df.isnull().mean()]
m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()

,count,%
Base Salary,184.0,26.099291
fb_id,0.0,0.000000
npxG/Sh,0.0,0.000000
pAdj_total_pressing,0.0,0.000000
pAdj_Num_Dribbled_past,0.0,0.000000


In [30]:
#drop cols with negatives 

df_n = df.drop(columns= ['Base Salary','Player', 'Nation', 'Position','fb_id', 'player_name', 'Squad_from_op','G-xG','np:G-xG'])
df_n.shape

(705, 100)

In [29]:
df_n.columns.values

array(['Opposing_Poss', 'Opposing_Touches', 'Tackle_pct_won', 'Nineties',
       'xA', 'Key_pass', 'Comp_prog', 'Total_Cmp', 'Total_Att',
       'Total_Cmp_pct', 'Total_Tot_Dist', 'Total_Prg_Dist', 'Short_Cmp',
       'Short_Att', 'Short_Cmp_pct', 'Medium_Cmp', 'Medium_Att',
       'Medium_Cmp_pct', 'Long_Cmp', 'Long_Att', 'Long_Cmp_pct',
       'touches', 'touches_def_box', 'touches_def_3rd', 'touches_mid_3rd',
       'touches_att_3rd', 'touches_att_box', 'touch_live', 'dribble_Succ',
       'dribble_Att', 'dribble_Succ_pct', 'num_dribble_past',
       'dribble_megs', 'carries', 'carries_dist', 'carries_prog_dist',
       'carries_prog', 'carries_att_3rd', 'carries_att_box',
       'miss_control', 'dispossed', 'recep_targ', 'recep_succ',
       'recept_pct', 'Fouls_drawn', 'Offsides', 'Crosses', 'PK_won',
       'Aerial_win_pct', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90',
       'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG', 'npxG',
       'npxG/Sh', 'Base Salary', 'pAdj_Tota

In [14]:
df_n = df_n[['pAdj_def_interceptions', 'pAdj_from_mish_tackles_won',
       'pAdj_PK_conceded', 'pAdj_OG', 'pAdj_Recoveries',
       'pAdj_Aerial_Duels_lost', 'lost_tackles', 'pAdj_Tackle_int_blocks',
       'avg_shrt+med_pass_pct', 'pct_long_balls',
       'prog_carry+lng_comp+crosses', 'attacking_touches']]

In [42]:
## this is double scaled so not good 


errors = {}
components_dict = {}
for c in range(4,12):
    X = df_n  # reading the score data
    
    c = c
    # Create a MaxAbsScaler: scaler
    transformer = MaxAbsScaler().fit(X)

    scaled_X= transformer.transform(X)
    
    # Create an NMF model: nmf
    
    transformer = Normalizer().fit(scaled_X)
    normed_X= transformer.transform(scaled_X)
    
    nmf = NMF(n_components=c,max_iter=100000,init='nndsvda',alpha=.1, l1_ratio=.5, random_state=0)
   


    W = nmf.fit_transform(normed_X)
    H = nmf.components_
    err = nmf.reconstruction_err_
    print(nmf,'error :',err, '\n', ' W shape',W.shape,'H shape ',H.shape)
    errors[c]=err
    # Create a DataFrame: components_df
    components_df = pd.DataFrame(nmf.components_,columns=df_n.columns)
    # put data frame in dictionary to observe 
    components_dict[c]=components_df


NMF(alpha=0.1, init='nndsvda', l1_ratio=0.5, max_iter=100000, n_components=4,
    random_state=0) error : 6.935424739660231 
  W shape (705, 4) H shape  (4, 100)
NMF(alpha=0.1, init='nndsvda', l1_ratio=0.5, max_iter=100000, n_components=5,
    random_state=0) error : 6.628425445126606 
  W shape (705, 5) H shape  (5, 100)
NMF(alpha=0.1, init='nndsvda', l1_ratio=0.5, max_iter=100000, n_components=6,
    random_state=0) error : 6.381982445883418 
  W shape (705, 6) H shape  (6, 100)
NMF(alpha=0.1, init='nndsvda', l1_ratio=0.5, max_iter=100000, n_components=7,
    random_state=0) error : 6.260549146165027 
  W shape (705, 7) H shape  (7, 100)
NMF(alpha=0.1, init='nndsvda', l1_ratio=0.5, max_iter=100000, n_components=8,
    random_state=0) error : 6.038865447414664 
  W shape (705, 8) H shape  (8, 100)
NMF(alpha=0.1, init='nndsvda', l1_ratio=0.5, max_iter=100000, n_components=9,
    random_state=0) error : 5.982570774970583 
  W shape (705, 9) H shape  (9, 100)
NMF(alpha=0.1, init='nndsvda

# looking for coherent groups of features inside of components to pick c  

In [23]:
for i in components_dict:
    print('This is nmf with :',i, ' components')
    print(' error = ',errors[i])
    for a in range(len(components_dict[i])):
        component = components_dict[i].iloc[a]
        print(component.nlargest())
    print('\n')

This is nmf with : 4  components
 error =  4.596288813307577
avg_shrt+med_pass_pct          4.454749
pct_long_balls                 2.280901
prog_carry+lng_comp+crosses    1.288301
pAdj_Recoveries                0.976971
pAdj_PK_conceded               0.070610
Name: 0, dtype: float64
pAdj_def_interceptions         2.668022
pAdj_Tackle_int_blocks         2.411153
pAdj_Recoveries                1.561454
prog_carry+lng_comp+crosses    1.371265
pAdj_from_mish_tackles_won     1.288471
Name: 1, dtype: float64
avg_shrt+med_pass_pct          2.364094
attacking_touches              2.257160
prog_carry+lng_comp+crosses    1.444761
pAdj_Recoveries                0.724509
lost_tackles                   0.417975
Name: 2, dtype: float64
avg_shrt+med_pass_pct     1.170728
pAdj_Aerial_Duels_lost    1.055999
pAdj_Recoveries           0.230715
attacking_touches         0.211670
pAdj_Tackle_int_blocks    0.185520
Name: 3, dtype: float64


This is nmf with : 5  components
 error =  3.672639771965719
avg_s

In [32]:

X = df_n  # reading the score data

c = 9
# Create a MaxAbsScaler: scaler
transformer = MaxAbsScaler().fit(X)

scaled_X= transformer.transform(X)

# Create an NMF model: nmf

nmf = NMF(n_components=c,max_iter=100000,init='nndsvda', random_state=0)
print(nmf)


W = nmf.fit_transform(scaled_X)
H = nmf.components_
err = nmf.reconstruction_err_

# Create a DataFrame: components_df
components_df = pd.DataFrame(nmf.components_,columns=df_n.columns)


NMF(init='nndsvda', max_iter=100000, n_components=9, random_state=0)


In [33]:
err

20.008525403805137

In [34]:
for i in range(len(components_df)):
    component = components_df.iloc[i]
    print(component.nlargest())

Opposing_Poss            6.987499
Opposing_Touches         6.955650
Short_Cmp_pct            6.889441
avg_shrt+med_pass_pct    6.706971
Medium_Cmp_pct           6.524501
Name: 0, dtype: float64
SoT%                2.964547
Offsides            2.866132
Sh                  2.674760
Opposing_Poss       2.640646
Opposing_Touches    2.638694
Name: 1, dtype: float64
Nineties                 2.892349
Opposing_Poss            1.175747
Opposing_Touches         1.139518
Short_Cmp_pct            1.095438
avg_shrt+med_pass_pct    1.074798
Name: 2, dtype: float64
Total_Att         1.453587
Total_Cmp         1.419352
recep_succ        1.379148
Total_Tot_Dist    1.302995
Medium_Att        1.269503
Name: 3, dtype: float64
pAdj_pres_Mid_3rd         2.491105
pAdj_total_pressing       2.463898
pAdj_Won_ball_5_sec       2.045346
pAdj_Fouls                1.623518
pAdj_Num_Dribbled_past    1.588285
Name: 4, dtype: float64
dribble_Succ        1.385609
num_dribble_past    1.290815
dispossed           1.03562

In [35]:
transformer = Normalizer().fit(W)
normed_X= transformer.transform(W)
normed_df = pd.DataFrame(normed_X,index=df.player_name.values)
normed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 705 entries, Carles Gil to Kamron Habibullah
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       705 non-null    float64
 1   1       705 non-null    float64
 2   2       705 non-null    float64
 3   3       705 non-null    float64
 4   4       705 non-null    float64
 5   5       705 non-null    float64
 6   6       705 non-null    float64
 7   7       705 non-null    float64
 8   8       705 non-null    float64
dtypes: float64(9)
memory usage: 55.1+ KB


In [36]:
normed_df.head(3)

,0,1,2,3,4,5,6,7,8
Carles Gil,0.003094,0.060928,0.389101,0.580208,0.000905,0.674940,0.000000,0.204739,0.103798
Julian Gressel,0.000000,0.060970,0.452329,0.463958,0.000000,0.271189,0.101103,0.256814,0.653227
Emanuel Reynoso,0.000000,0.067666,0.310373,0.388408,0.160037,0.804677,0.000000,0.050029,0.269463


In [37]:
tt = normed_df.loc['Tommy Thompson']
sims = normed_df.dot(tt)

sims.nlargest(10)

Tommy Thompson     1.000000
Emmanuel Más       0.992373
Oniel Fisher       0.991727
Jaylin Lindsey     0.988957
Edgar Castillo     0.979589
Eddie Munjoma      0.978418
Tayvon Gray        0.975577
Niko Hämäläinen    0.974743
Zico Bailey        0.972104
D.J. Taylor        0.971808
dtype: float64

In [38]:
print([i for i in df['player_name'] if 'Walker' in i ])

['Walker Zimmerman']


In [39]:
wz = normed_df.loc['Walker Zimmerman']
sims = normed_df.dot(wz)

sims.nlargest(10)

Walker Zimmerman     1.000000
Ranko Veselinović    0.995950
Tanner Beason        0.994726
Xavier Arreaga       0.992998
Bill Tuiloma         0.992017
Oswaldo Alanís       0.991455
Jack Maher           0.991207
Larrys Mabiala       0.988112
Robin Jansson        0.987766
Nkosi Burgess        0.987288
dtype: float64

In [40]:
jm = normed_df.loc['Jonathan Mensah']
sims = normed_df.dot(jm)

sims.nlargest(10)

Jonathan Mensah      1.000000
Antonio Carlos       0.996214
Rudy Camacho         0.994820
Andy Rose            0.993489
Larrys Mabiala       0.993116
Julio Cascante       0.993039
Josh Williams        0.989379
Nkosi Burgess        0.988423
Matt Hedges          0.986246
Ranko Veselinović    0.984544
dtype: float64

## need to test results with less features to see if error goes down 